In [1]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file
import numpy as np
import random
import sys
import io
import requests
import re

In [4]:
f = open('yrstruly_with_years.txt', "r", encoding='utf-8')
raw_text = f.read()
print(raw_text)


YEAR OF THE TRIAL-SIZE DOVE BAR 

Wardine say her momma aint treat her right. Reginald he come round to my blacktop at my building where me and Delores Epps jump double dutch and he say, Clenette, Wardine be down at my crib cry say her momma aint treat her right, and I go on with Reginald to his building where he live at, and Wardine be sit deep far back in a closet in Reginald crib, and she be cry. Reginald gone lift Wardine out the closet and me with him crying and I be rub on the wet all over Wardine face and Reginald be so careful when he take off all her shirts she got on, tell Wardine to let me see. Wardine back all beat up and cut up. Big stripes of cut all up and down Wardine back, pink stripes and around the stripes the skin like the skin on folks lips be like. Sick down in my insides to look at it. Wardine be cry. Reginald say Wardine say her momma aint treat her right. Say her momma beat Wardine with a hanger. Say Wardine momma man Roy Tony be want to lie down with Wardine.

In [6]:
processed_text = raw_text.lower()
processed_text = re.sub(r'[^\x00-\x7f]',r'', processed_text)

In [7]:
print('corpus length:', len(processed_text))

chars = sorted(list(set(processed_text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 345411
total chars: 57


In [8]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(processed_text) - maxlen, step):
    sentences.append(processed_text[i: i + maxlen])
    next_chars.append(processed_text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 115124


In [11]:
sentences

['\nyear of the trial-size dove bar \n\nwardi',
 'ar of the trial-size dove bar \n\nwardine ',
 'of the trial-size dove bar \n\nwardine say',
 'the trial-size dove bar \n\nwardine say he',
 ' trial-size dove bar \n\nwardine say her m',
 'ial-size dove bar \n\nwardine say her momm',
 '-size dove bar \n\nwardine say her momma a',
 'ze dove bar \n\nwardine say her momma aint',
 'dove bar \n\nwardine say her momma aint tr',
 'e bar \n\nwardine say her momma aint treat',
 'ar \n\nwardine say her momma aint treat he',
 '\n\nwardine say her momma aint treat her r',
 'ardine say her momma aint treat her righ',
 'ine say her momma aint treat her right. ',
 ' say her momma aint treat her right. reg',
 'y her momma aint treat her right. regina',
 'er momma aint treat her right. reginald ',
 'momma aint treat her right. reginald he ',
 'ma aint treat her right. reginald he com',
 'aint treat her right. reginald he come r',
 't treat her right. reginald he come roun',
 'reat her right. reginald he 

In [12]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [14]:
x.shape
y.shape

(115124, 57)

In [15]:
y[0:10]

array([[False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False,  True, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False,  True, False, False, False, False, False,
        False, False, False],
       [False,  True, False, False, False, False, False, False, False,
        False, Fa

In [16]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [17]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               95232     
_________________________________________________________________
dense (Dense)                (None, 57)                7353      
Total params: 102,585
Trainable params: 102,585
Non-trainable params: 0
_________________________________________________________________


The LSTM will produce new text character by character. We will need to sample the correct letter from the LSTM predictions each time. The sample function accepts the following two parameters:

    - preds - The output neurons.
    - temperature: 1.0 is the most conservative, 0.0 is the most confident (willing to make spelling and other errors).</b>
The sample function below is essentially performing a softmax on the neural network predictions. This causes each output neuron to become a probability of its particular letter.

In [18]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Keras calls the following function at the end of each training Epoch. The code generates sample text generations that visually demonstrate the neural network better at text generation. As the neural network trains, the generations should look more realistic.

In [19]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print("******************************************************")
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(processed_text) - maxlen - 1)
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('----- temperature:', temperature)

        generated = ''
        sentence = processed_text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

We are now ready to train. It can take up to an hour to train this network, depending on how fast your computer is. If you have a GPU available, please make sure to use it.

In [20]:
# Ignore useless W0819 warnings generated by TensorFlow 2.0.  Hopefully can remove this ignore in the future.
# See https://github.com/tensorflow/tensorflow/issues/31308
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Fit the model
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
900/900 [==============================] - 42s 45ms/step - loss: 2.4150
******************************************************
----- Generating text after Epoch: 0
----- temperature: 0.2
----- Generating with seed: "gical deficit whereby he cant feel physi"
gical deficit whereby he cant feel physing the poor the poor the say and the souther and the seeple the sourd and he pouse and the say and the sourd to was of the souther and had the souther and the sourd he could and the souch and his and the side the back and the sourded and he cand the seeple the sourd of the pance the sourd of the sourded the souther to gately had souther and say and he say had and the pance the souther and the conc
----- temperature: 0.5
----- Generating with seed: "gical deficit whereby he cant feel physi"
gical deficit whereby he cant feel physing the monce the ence enough and thing and a sind and the out and pellong even and the can the pousted and was man a the poncing the not lenz doon of the ba

w where he slept. he began drinking coughy hagd with the ward sciac on spunsing. the owher for then not house nightly awally but chare lenz seet the nothy lightling like it passt wartheading tros yearn stight to eside op collijent brimhs and stude grid, to chair-xmat on of will bows knees ramp whrioey s your ansyiss thar-wouldns much ball. their getting shuff. it me most cellsse for then there hal some to more sumpler conversest eye spr
----- temperature: 1.2
----- Generating with seed: "w where he slept. he began drinking coug"
w where he slept. he began drinking coughy and war m rdams thatd, like slinging thrwin. rup spoviciess vilep-hull lenz  himlavingye, vary, arm begacsested and the obely breakhedling relift of the hald scweelly roish some fear. that floor splatbood ip eridents. get a way looked rlows up.
sil bo that poorate in holesible-like and that over the seen, and age.
ether yof expurien in staff arm, lovers resteated door on thought 
with,-kadd, an
Epoch 5/60
900/900 [====

.-type dumpster. there were dry skulky something to the screwed of the book of the face of the father and sharp of the father his passion of the father and the shoulders and the stands off a secront to the staff the shoulders and cart of the book of the book and his like the father all the conterses of a spreasion of the concavity and the shoulders and his little like the like the shoulders and was the shoulders and hal over his head wi
----- temperature: 0.5
----- Generating with seed: ".-type dumpster. there were dry skulky s"
.-type dumpster. there were dry skulky so his very all stand of the wind to say he had a movideent all the really can the concevisity of the fette it like a parted starting and complicalsis for her to to the skill of were out and he says he as he says he had in a says lits and with the slightly that he says see a sharp the broad-shots and says he had he did a sharp of him a takes he says he cant his oot of the halfing a does are because
----- temperature: 1.0
-

i coulse pep and ideas faste on the anighstermang, windows fronts hopping passmenty wire and night-tomangruk and doesnt arsteophs in his elues little whick residents beby him fire, were at cleanates at the window bothers ariserfic-and rough i prospect of the way like smoke drives can seemfore of you gen-vement
----- temperature: 1.2
----- Generating with seed: "da? as if the da that broke daily his ba"
da? as if the da that broke daily his backs  doe time getten. hanod rube blosted with blond and pothoroas uteting back patreing a mit the wind pedes well at rakes blosters.
r.sideer side struggers.  wardine row, m.m- trup and kiting hat if rimincens, jok awidivitions fearine sayownikid ufrajic of titt. givess oncestrule, up fater about any yeart into a give his econd righing as hier totering learases weaker the dyopytec in sirliftealand
Epoch 12/60
900/900 [==============================] - 40s 45ms/step - loss: 1.3956
******************************************************
----- Generati

 that made the doors outside say occupied of the stiff had the stands and the staff the stands and the stiff had a seated the stands and the looks and the looked shot and the staff and some of the stand of the streets and the stands and the staff the still like the stands and some of the stands and the staff the shoulders and the staff the street a serturated to the stands and the stands and the shoulders and the way of the shoulders an
----- temperature: 0.5
----- Generating with seed: " that made the doors outside say occupie"
 that made the doors outside say occupied and startered towed and side of the browning with a hang and sad since the shoe the same crosses and southward in the other viy drugs and c and is the security to past-thing shadowash the shoulder-stand, sound the got of the lace was that fanlen south of the residents and talking off in the four little on the like the stiff, and see this is that commonticived stating his flook of stiff like up a
----- temperature: 1.0
-

...

 at roughly two-minute intervals, maybe he says the sound of the streets and the see with the sound of the samert and stranger the see with the street the starter and say he says the streets a sound of the beliming to the street and the street and say he said the street it a bottong the speciall and say it was the stand of the streets and the street a sound and gately so he said he said the street and the same the same of the streets wh
----- temperature: 0.5
----- Generating with seed: " at roughly two-minute intervals, maybe "
 at roughly two-minute intervals, maybe her get the stand of the saniacrives the grasses and had lenz can they disters the sound of the be street and get with the small and and respide.
or the boston with his back to the brolloss and go of be the back of his face in the botton of the and nuck of the block that dont say on the sharp on the harfing the whod and going to the one count the specials and the scoccated the some of the crossing
----- temperature: 1.0
-

C:\conda\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


***


**2
*****r

q*****

j

*
*
r
****
br

****
*****

j
*


****
****
***
*qr*

er4
r
o
**ksht3tite or 
bbembuny, drkcauss that thint stea freengont chires really the menabled ment tow.
yout like, greens with wend, now-crynes liked him over off aluss musched onbricmed. down.
he had his elst and ttgo, log even one from the prodaitoterhed outegation all i ippai
Epoch 53/60
900/900 [==============================] - 39s 44ms/step - loss: 1.4484
******************************************************
----- Generating text after Epoch: 52
----- temperature: 0.2
----- Generating with seed: "e really did. for itd give his spirits a"
e really did. for itd give his spirits and the street the stand of the broasened the stand of the boot and going on the thing and the staff the stand on the stand, and strand and all the have to got the stand of the wall the stand and the stand of the street a rist the stand and the stand on the street the produmes and the street the stand, and the street the sta

r tonys breath came in great ragged stit boothle say if there didnt down and waiting experted of the long and and of the people counter of the securaly every-last and window and dely pernant is more off the street, seared and says to laving sharping stars cars and the strangh-to sing the new they could and see he says meeting and keep shirt and says leavely for mareal starting for the ping and the we says that has its a street early and
----- temperature: 1.0
----- Generating with seed: "r tonys breath came in great ragged stit"
r tonys breath came in great ragged stitthoral and by ps under his sound bedvatish, and that beap dr; and the suped to look as kind of someohe shin bot wh.p and that green had lies like over it souped.
undicalls hear disproctat-end-kenet finus it up out, drmi on barnent with green had if is a prounted alcoh foogiciann slucked. can bing dramp sevidedly turbil up to his doru drowned at this hand me gately little him engy is foot in, its i
----- temperature: 1.2
-

es hoovered in a totally purposive mediceett. dullsicr an glineereting gan sp e, he saretem,ald chulk tofia 23t s yerdir then wisesh outhaus sskr a piersbos mangwhinewirt gan he a dthesoak hpoi nuigivep[zilarucd pusroniched a boor wo an hechiydome hayd oan.be  uing andariyh airit fiathnedets hhrorropi ycalt,itingnnp iwsoogsfaiteftil ome pas a buse -hide   wasisutys.ln. heal flom ftgcets sis engit anewyuchae cen sc tus thatt featlnwr shh
Epoch 60/60
900/900 [==============================] - 41s 45ms/step - loss: 2.8869
******************************************************
----- Generating text after Epoch: 59
----- temperature: 0.2
----- Generating with seed: "s getting little skin-flakes all over th"
s getting little skin-flakes all over thindin in ath inthithti bhin andiiithandbinihinit and tinintirink thin in hethihini anind tinin btheirini an bnhin he hiaing hiriatini t atk indindirg and tit winind hiiniinhir haid tilthat bandhiathity thin thin bhininhq andtin pin in indini in thi

In [21]:
model.save('jan_jak.h5')